In [112]:
pip install torch tensorflow transformers datasets

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached torch-2.2.2-cp39-none-macosx_10_9_x86_64.whl.metadata (25 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.8/150.8 MB 59.6 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
import torch
print(torch.cuda.is_available())

False


In [92]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_name = "distilbert-base-uncased"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [93]:
import pandas as pd

In [23]:
imdb_data = pd.read_csv('IMDB Dataset.csv')
print(imdb_data.shape)
imdb_data.head(10)

positive = imdb_data[imdb_data['sentiment'] == "positive"]
negative = imdb_data[imdb_data['sentiment'] == "negative"]

positive_sample = positive.sample(n=100, random_state=42)
negative_sample = negative.sample(n=100, random_state=42)
small_dataset = pd.concat([positive_sample, negative_sample])

(50000, 2)


In [24]:
imdb_data.describe()
small_dataset.describe()

,review,sentiment
count,200,200
unique,200,2
top,I don't know how or why this film has a meager...,positive
freq,1,100


In [25]:
small_dataset['sentiment'].value_counts()

sentiment
positive    100
negative    100
Name: count, dtype: int64

In [32]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    small_dataset['review'],  # Features (reviews)
    small_dataset['sentiment'], # Target (sentiment)
    test_size=0.2,             # 80% train, 20% test
    random_state=42,           # For reproducibility
    stratify=small_dataset['sentiment']  # Ensure balanced sentiment classes
)

# Check the class distribution in both train and test sets
print("Training set class distribution:")
print(y_train.value_counts())
print("\nTest set class distribution:")
print(y_test.value_counts())


Training set class distribution:
sentiment
negative    80
positive    80
Name: count, dtype: int64

Test set class distribution:
sentiment
negative    20
positive    20
Name: count, dtype: int64


In [94]:
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, Trainer, TrainingArguments, AutoModelForSequenceClassification
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import concatenate_datasets


# Step 1: Load the IMDB dataset from a local CSV file
dataset = load_dataset("csv", data_files={"train": "IMDB Dataset.csv"})

# Step 2: Inspect the dataset structure (check the column names)
print(dataset['train'].column_names)

# Step 3: Assuming the sentiment column is named 'sentiment' and the review column is 'review',
# We filter the dataset to get 100 positive and 100 negative samples

positive_samples = dataset['train'].filter(lambda x: x['sentiment'] == 'positive')
negative_samples = dataset['train'].filter(lambda x: x['sentiment'] == 'negative')

# Shuffle and sample 100 from each class
positive_sample = positive_samples.shuffle(seed=42).select([i for i in range(100)])
negative_sample = negative_samples.shuffle(seed=42).select([i for i in range(100)])

small_dataset = concatenate_datasets([positive_sample, negative_sample])

# Step 4: Split the dataset into train and test (80% train, 20% test)
train_df, test_df = train_test_split(
    small_dataset.to_pandas(), 
    test_size=0.2, 
    stratify=small_dataset['sentiment'],  # Stratified split to maintain balance
    random_state=42
)

# Step 5: Check class distribution in the training and test sets
print("Training set class distribution:")
print(train_df['sentiment'].value_counts())

print("\nTest set class distribution:")
print(test_df['sentiment'].value_counts())

# Step 6: Preprocess the dataset - Tokenize the data
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def preprocess_function(examples):
    return tokenizer(examples['review'], truncation=True, padding=True)

# Convert the pandas dataframes back to dataset objects
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Tokenize the datasets
tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)
tokenized_test_dataset = test_dataset.map(preprocess_function, batched=True)

# # Step 7: Define training arguments
# training_args = TrainingArguments(
#     output_dir="./results",
#     evaluation_strategy="epoch",
#     learning_rate=2e-5,
#     per_device_train_batch_size=16,
#     num_train_epochs=3,
#     weight_decay=0.01,
# )

# # Load a model for sequence classification (adjust for your specific use case)
# model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# # Step 8: Define Trainer and Train the Model
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=tokenized_train_dataset,
#     eval_dataset=tokenized_test_dataset,  # Use the test set for evaluation
# )

['review', 'sentiment']
Training set class distribution:
sentiment
negative    80
positive    80
Name: count, dtype: int64

Test set class distribution:
sentiment
negative    20
positive    20
Name: count, dtype: int64


Map: 100%|██████████| 40/40 [00:00<00:00, 487.44 examples/s]


In [2]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
from datasets import load_dataset, Dataset, concatenate_datasets
import pandas as pd
from sklearn.model_selection import train_test_split

# Load model and tokenizer
model_name = "distilbert-base-uncased"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load the IMDB dataset from a local CSV file
dataset = load_dataset("csv", data_files={"train": "IMDB Dataset.csv"})

# Inspect the dataset structure (check the column names)
print(dataset['train'].column_names)

# Filter for positive and negative samples
positive_samples = dataset['train'].filter(lambda x: x['sentiment'] == 'positive')
negative_samples = dataset['train'].filter(lambda x: x['sentiment'] == 'negative')

# Shuffle and sample 100 from each class
positive_sample = positive_samples.shuffle(seed=42).select([i for i in range(100)])
negative_sample = negative_samples.shuffle(seed=42).select([i for i in range(100)])

# Concatenate positive and negative samples
small_dataset = concatenate_datasets([positive_sample, negative_sample])

# Convert sentiment column to numerical labels (0 for negative, 1 for positive)
small_dataset = small_dataset.map(
    lambda x: {"label": [1 if sentiment == 'positive' else 0 for sentiment in x['sentiment']]}, 
    batched=True
)

# Split the dataset into train and test sets (80% train, 20% test)
train_df, test_df = train_test_split(
    small_dataset.to_pandas(), 
    test_size=0.2, 
    stratify=small_dataset['sentiment'],  # Stratified split to maintain balance
    random_state=42
)

# Convert the pandas dataframes back to dataset objects
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Tokenize the datasets
def preprocess_function(examples):
    return tokenizer(examples['review'], truncation=True, padding=True)

tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)
tokenized_test_dataset = test_dataset.map(preprocess_function, batched=True)

# Define the training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
)

# Start training
trainer.train()


/Users/albertwelong/cognizant_genai/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/albertwelong/cognizant_genai/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-08 23:36:00.265537: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased an

['review', 'sentiment']


Map: 100%|██████████| 40/40 [00:00<00:00, 693.24 examples/s]
/Users/albertwelong/cognizant_genai/.venv/lib/python3.9/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,0.687670
2,No log,0.680560
3,No log,0.677075


TrainOutput(global_step=30, training_loss=0.6751163482666016, metrics={'train_runtime': 1600.949, 'train_samples_per_second': 0.3, 'train_steps_per_second': 0.019, 'total_flos': 63584351354880.0, 'train_loss': 0.6751163482666016, 'epoch': 3.0})

In [7]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
from datasets import load_dataset, Dataset, concatenate_datasets
from sklearn.model_selection import train_test_split

# Load model and tokenizer
model_name = "distilbert-base-uncased"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load the IMDB dataset from a local CSV file
dataset = load_dataset("csv", data_files={"train": "IMDB Dataset.csv"})

# Inspect the dataset structure (check the column names)
print(dataset['train'].column_names)

# Filter for positive and negative samples
positive_samples = dataset['train'].filter(lambda x: x['sentiment'] == 'positive')
negative_samples = dataset['train'].filter(lambda x: x['sentiment'] == 'negative')

# Shuffle and sample 100 from each class
positive_sample = positive_samples.shuffle(seed=42).select([i for i in range(100)])
negative_sample = negative_samples.shuffle(seed=42).select([i for i in range(100)])

# Concatenate positive and negative samples
small_dataset = concatenate_datasets([positive_sample, negative_sample])

# Convert sentiment column to numerical labels (0 for negative, 1 for positive)
def sentiment_to_label(example):
    example['label'] = 1 if example['sentiment'] == 'positive' else 0
    return example

small_dataset = small_dataset.map(sentiment_to_label)

# Split the dataset into train and test sets (80% train, 20% test)
train_df, test_df = train_test_split(
    small_dataset.to_pandas(), 
    test_size=0.2, 
    stratify=small_dataset['label'],  # Stratified split to maintain balance
    random_state=42
)

# Convert the pandas dataframes back to dataset objects
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Tokenize the datasets
def preprocess_function(examples):
    return tokenizer(examples['review'], truncation=True, padding=True)

tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)
tokenized_test_dataset = test_dataset.map(preprocess_function, batched=True)

# Define the training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
)

# Start training
trainer.train()


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


['review', 'sentiment']


Map: 100%|██████████| 40/40 [00:00<00:00, 342.90 examples/s]
/Users/albertwelong/cognizant_genai/.venv/lib/python3.9/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,0.683228
2,No log,0.673575
3,No log,0.670827


TrainOutput(global_step=30, training_loss=0.6682209650675456, metrics={'train_runtime': 1185.0091, 'train_samples_per_second': 0.405, 'train_steps_per_second': 0.025, 'total_flos': 63584351354880.0, 'train_loss': 0.6682209650675456, 'epoch': 3.0})

In [3]:
model.save_pretrained("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")

('./fine_tuned_model/tokenizer_config.json',
 './fine_tuned_model/special_tokens_map.json',
 './fine_tuned_model/vocab.txt',
 './fine_tuned_model/added_tokens.json',
 './fine_tuned_model/tokenizer.json')

In [4]:
results = trainer.evaluate()
print(results)

{'eval_loss': 0.6770753860473633, 'eval_runtime': 26.5213, 'eval_samples_per_second': 1.508, 'eval_steps_per_second': 0.189, 'epoch': 3.0}


In [8]:
results = trainer.evaluate()
print(results)

{'eval_loss': 0.6708268523216248, 'eval_runtime': 21.6157, 'eval_samples_per_second': 1.851, 'eval_steps_per_second': 0.231, 'epoch': 3.0}


In [6]:
from sklearn.metrics import classification_report
predictions = trainer.predict(tokenized_test_dataset)
y_pred = predictions.predictions.argmax(axis=1)
y_true = tokenized_test_dataset["label"]
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.58      0.90      0.71        20
           1       0.78      0.35      0.48        20

    accuracy                           0.62        40
   macro avg       0.68      0.62      0.59        40
weighted avg       0.68      0.62      0.59        40



In [9]:
from sklearn.metrics import classification_report
predictions = trainer.predict(tokenized_test_dataset)
y_pred = predictions.predictions.argmax(axis=1)
y_true = tokenized_test_dataset["label"]
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.61      0.95      0.75        20
           1       0.89      0.40      0.55        20

    accuracy                           0.68        40
   macro avg       0.75      0.68      0.65        40
weighted avg       0.75      0.68      0.65        40



In [10]:
from sklearn.metrics import classification_report
predictions = trainer.predict(tokenized_test_dataset)
y_pred = predictions.predictions.argmax(axis=1)
y_true = tokenized_test_dataset["label"]
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.61      0.95      0.75        20
           1       0.89      0.40      0.55        20

    accuracy                           0.68        40
   macro avg       0.75      0.68      0.65        40
weighted avg       0.75      0.68      0.65        40

